In [ ]:
#import
import pandas as pd
from pprint import pprint
import json
import random

In [ ]:
#mount to googledrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#file path
root = '/content/drive/My Drive/Swinburne/TIP/data/'


In [ ]:
input_train_SciQ = root + 'train_SciQ.json'
output_train_SciQ = root + 'train_SciQ.jsonl'

with open(input_train_SciQ, 'r') as infile:
    data = json.load(infile)

# Write the data to the output JSONL file
with open(output_train_SciQ, 'w') as outfile:
    for item in data:
        outfile.write(json.dumps(item) + '\n')

with open(output_train_SciQ, 'r') as f:
    lines = f.readlines()
    count = len(lines)

print(f"There are {count} items in train_SciQ.jsonl file")


There are 11679 items in train_SciQ.jsonl file


In [ ]:
file_paths = ['ARC-Easy-Train.jsonl','common_sense_train.jsonl','dev_rand_split.jsonl','train_SciQ.jsonl']

# Update file paths to include the root directory
for i in range(len(file_paths)):
  file_paths[i] = root + file_paths[i]
output_file_path = root + 'cleaned_ARC_CommonSense_SciQ.jsonl' 

In [ ]:
import json
import random

# DICTIONARY TEMPLATE
# Create a template dictionary for the questions
dic = {}
dic['id'] = 'x'
dic_question = {}
dic_question['stem'] = 'xx'
dic_choices = [0] * 4
dic_choices[0] = {'text': 'ada', 'label': 'A'}
dic_choices[1] = {'text': 'ada', 'label': 'B'}
dic_choices[2] = {'text': 'ada', 'label': 'C'}
dic_choices[3] = {'text': 'ada', 'label': 'D'}
dic_question['choices'] = dic_choices
dic['question'] = dic_question
dic['answerKey'] = 'D'
mcq = {0: "A", 1: "B", 2: "C", 3: "D", 4: "E", '0': 'A', '1': 'B', '2': 'C', '3': 'D', '4': "E"}

In [ ]:
# OUTPUT FILE
fout = open(output_file_path, 'wt')

# PROCESSING FILES
for file_path in file_paths:
    with open(file_path) as json_file:
        json_list = list(json_file)
        for i in range(len(json_list)):
            json_str = json_list[i].strip()  # Remove leading and trailing whitespaces
            if not json_str:  # Skip empty lines
                continue
            try:
                result = json.loads(json_str)
            except json.JSONDecodeError:
                print(f"Error decoding JSON on line {i+1} in file {file_path}: {json_str}")
                continue

            if 'distractor1' in result:
                # Convert the new format to the expected format
                converted_result = {
                    'question': {
                        'stem': result['question'],
                        'choices': [
                            {'text': result['distractor1'], 'label': 'A'},
                            {'text': result['distractor2'], 'label': 'B'},
                            {'text': result['distractor3'], 'label': 'C'},
                            {'text': result['correct_answer'], 'label': 'D'}
                        ]
                    },
                    'answerKey': 'D'
                }
                result = converted_result

            # Check if 'question' is a dictionary and if 'choices' key exists
            if isinstance(result['question'], dict) and 'choices' in result['question'] and len(result['question']['choices']) < 4:
                continue

            # Populate the template dictionary with data from the current question
            dic['question'] = result['question']
            dic['question']['choices'][0]['text'] = result['question']['choices'][0]['text']
            dic['question']['choices'][1]['text'] = result['question']['choices'][1]['text']
            dic['question']['choices'][2]['text'] = result['question']['choices'][2]['text']
            dic['question']['choices'][3]['text'] = result['question']['choices'][3]['text']
            dic['answerKey'] = result['answerKey']

            # Remap answer keys if they are in numeric format
            if dic['answerKey'] not in ('A', 'B', 'C', 'D', 'E'):
                dic['answerKey'] = mcq.get(dic['answerKey'], dic['answerKey'])

            # Handle questions with answer key 'E'
            if dic['answerKey'] == 'E' and len(result['question']['choices']) > 4:
                new_answer_choice = random.randint(0, 3)
                dic['question']['choices'][new_answer_choice]['text'] = result['question']['choices'][4]['text']
                dic['answerKey'] = mcq[new_answer_choice]

            # Write the modified dictionary to the output file
            fout.write('%s\n' % json.dumps(dic))

fout.close()

In [ ]:
print(result['question'])

{'stem': 'What do you call a structure composed of two or more types of tissues that work together to do a specific task?', 'choices': [{'text': 'cell', 'label': 'A'}, {'text': 'marrow', 'label': 'B'}, {'text': 'system', 'label': 'C'}, {'text': 'organ', 'label': 'D'}]}


In [ ]:
with open(output_file_path, 'r') as f:
    lines = f.readlines()
    count = len(lines)

print(f"There are {count} items in the final file.")


There are 16366 items in the final file.


In [ ]:
with open(output_file_path, 'r') as f:
    lines = f.readlines()
    count = len(lines)

print(f"There are {count} lines in cleaned_ARC_CommonSense_SciQ.jsonl.")


There are 16366 lines in cleaned_ARC_CommonSense_SciQ.jsonl.
